### Методика подготовки кастомизированного Allowed List на основе файла от Кастомера

In [72]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import sys
from os.path import join, normpath
import os
import roamability as rb
import re
from collections import namedtuple

#######################################################################################
# Define variables here
#######################################################################################

# Input files
downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA'
dmi_allowed_list = 'DMI.dmi_allowed_list_export_Wed_Jan_22_2020_after.csv'
dmi_sts_list = 'Steering.sts_sponsor_country_export_Mon_Jan_06_2020.csv'

file_new_al = 'AL_SMART_ALL.csv' # File name with the new Allowed list
# The example of the content of the file with the list of allowed networks:
# TADIG,SPONSOR
# ALBVF,S1
# ARGCM,S5
# ARM01,S2
# AUSOP,S1
# AUTHU,S2

# Output files
out_file_not_disc = 'AlNotDiscounted.csv'
out_file_toc_al_delete_not_disc = 'AlNotDiscountedDelete.rcjson'

# Задать имена для создаваемых, кастомизированных Allowed list
# Правило задания имен: <ИмяСпоносра>_TEST_COMB
s1_al_name = ''
s2_al_name = ''
s4_al_name = ''
s5_al_name = 'SMART'
s6_al_name = ''
s8_al_name = ''

# Задать имена для создаваемых, кастомизированных [sts] Sponsor Profile
# Правило задания имен: S1_TEST_STS
s1_sts_profile_name = ''
s2_sts_profile_name = ''
s4_sts_profile_name = ''
s5_sts_profile_name = ''
s6_sts_profile_name = ''
s8_sts_profile_name = ''

# Задать имена для существующих "All" Allowed list (разрешены будут только те сети, которые присутствуют в 'All')
s1_al_all_name = 'Partner_All'
s2_al_all_name = 'P4_All'
s4_al_all_name = 'MB_ALL'
s5_al_all_name = 'SMART'
s6_al_all_name = 'S6_ALL'
s8_al_all_name = 'S8_ALL'

# Задать имена для существующих, основных "Combined" Allowed list
s1_al_combined_def_name = 'Partner_Combined'
s2_al_combined_def_name = 'P4_Combined'
s4_al_combined_def_name = 'MB_Combined'
s5_al_combined_def_name = 'SMART_Combined'
s6_al_combined_def_name = 'S6_Combined'
s8_al_combined_def_name = 'S8_Combined'

#######################################################################################

#######################################################################################

sponsor_object = namedtuple('Sponsor',
                'sponsor sponsor_id al_all_name al_pattern al_name_combined_custom al_name_combined_def sts_profile_name')

s1_sponsor = sponsor_object('S1', 1, s1_al_all_name, r'[Pp][Aa][rtner]?[\s\w-]*|^S1_[\s\w-]*',
                            s1_al_name, s1_al_combined_def_name, s1_sts_profile_name)
s2_sponsor = sponsor_object('S2', 2, s2_al_all_name, r'[Pp]4[\s\w-]*|^S2_[\s\w-]*',
                            s2_al_name, s2_al_combined_def_name, s2_sts_profile_name)
s4_sponsor = sponsor_object('S4', 4, s4_al_all_name, r'[Mm][Bb][\s\w-]*|^S4_[\s\w-]*',
                            s4_al_name, s4_al_combined_def_name, s4_sts_profile_name)
s5_sponsor = sponsor_object('S5', 5, s5_al_all_name, r'[Ss][Mm][Aa][Rr][Tt][\s\w-]*|^S5_[\s\w-]*',
                            s5_al_name, s5_al_combined_def_name, s5_sts_profile_name)
s6_sponsor = sponsor_object('S6', 6, s6_al_all_name, r'^S6_[\s\w-]*',
                            s6_al_name, s6_al_combined_def_name, s6_sts_profile_name)
s8_sponsor = sponsor_object('S8', 8, s8_al_all_name, r'^S8_[\s\w-]*',
                            s8_al_name, s8_al_combined_def_name, s8_sts_profile_name)

sponsors = [s1_sponsor, s2_sponsor, s4_sponsor, s5_sponsor, s6_sponsor, s8_sponsor]

def prepare_toc_to_create_al(df, downloads, result_file_name):
    """df[['SPONSOR','TADIG']], where SPONSOR - S1, S2..."""
    ouf=open(join(downloads, result_file_name), 'w')
    num_val = df.count()[0] - 1
    t1='{"params":{"sponsor":"'
    t2='","plmnCode":"'
    t3='","mo":"1","mt":"1","sms":"1","data":"1","moSec":"1","dataKb":"1"},"type":"create","caption":"dmi_allowed_list","objectId":"DMI.dmi_allowed_list","serviceId":"DMI"}'
    ouf.write('[')
    for i, [sponsor_name, tadig] in enumerate(df.values):
        for [sponsor, al_name_combined] in [[sponsor.sponsor, sponsor.al_name_combined_custom] for sponsor in sponsors]:
            if sponsor_name == sponsor:
                ouf.write(t1+al_name_combined+t2+tadig+t3)
                if i < num_val:
                    ouf.write(',')
    ouf.write(']')
    ouf.close()

def prepare_toc_to_delete_al(df, downloads, result_file_name):
    """df[['Sponsor','TADIG']], where Sponsor - Allowed list name like Partner_Combined, P4_Combined..."""
    ouf = open(join(downloads, result_file_name), 'w')
    num_val = df.count()[0] - 1
    t1='{"params":{"dataKb":"1","sponsor":"'
    t2='","mo":"1","data":"1","mt":"1","sms":"1","plmnCode":"'
    t3='","moSec":"1"},"type":"remove","caption":"dmi_allowed_list","objectId":"DMI.dmi_allowed_list","serviceId":"DMI"}'
    ouf.write('[')
    for i, [sponsor_name, tadig] in enumerate(df.values):
        ouf.write(t1 + sponsor_name + t2 + tadig + t3)
        if i < num_val:
            ouf.write(',')
    ouf.write(']')
    ouf.close()

def prepare_toc_to_create_sts(df, downloads, result_file_name):
    """df[['SPONSOR','MCC','COUNTRY_NAME']], where SPONSOR - S1, S2..."""
    ouf=open(join(downloads, result_file_name), 'w')
    num_val = df.count()[0]-1
    t1 = '{"params":{"sponsorProfile":"'
    t2 = '","countryName":"'
    t3 = '","mcc":"'
    t4 = '"},"type":"create","caption":"sts_sponsor_country","objectId":"Steering.sts_sponsor_country","serviceId":"Steering"}'
    ouf.write('[')
    for i, [sponsor_name, mcc, country] in enumerate(df.values):
        for [sponsor, sts_sponsor_name] in [[sponsor.sponsor, sponsor.sts_profile_name] for sponsor in sponsors]:
            if sponsor_name == sponsor:
                ouf.write(t1 + sts_sponsor_name + t2 + country + t3 + str(mcc) + t4)
                if i < num_val:
                    ouf.write(',')
    ouf.write(']')
    ouf.close()

def prepare_toc_to_delete_sts(df, downloads, result_file_name):
    """df[['SPONSOR','MCC','COUNTRY_NAME']], where SPONSOR - S1, S2..."""
    ouf=open(join(downloads, result_file_name), 'w')
    num_val = df.count()[0] - 1
    t1='{"params":{"sponsorProfile":"'
    t2='","countryName":"'
    t3='","mcc":"'
    t4='"},"type":"remove","caption":"sts_sponsor_country","objectId":"Steering.sts_sponsor_country","serviceId":"Steering"}'
    ouf.write('[')
    for i, [sponsor_name, mcc, country] in enumerate(df.values):
        for [sponsor, sts_sponsor_name] in [[sponsor.sponsor, sponsor.sts_profile_name] for sponsor in sponsors]:
            if sponsor_name == sponsor:
                ouf.write(t1 + sts_sponsor_name + t2 + country + t3 + str(mcc) + t4)
                if i < num_val:
                    ouf.write(',')
    ouf.write(']')
    ouf.close()

def check_al_for_differences(df_al_current, df_al_new):
    """df_al_current[['Sponsor','TADIG', 'SPONSOR']], df_al_new[['Sponsor','TADIG', 'SPONSOR']]"""
    df = pd.merge(df_al_current[['Sponsor','TADIG','SPONSOR']],
                    df_al_new[['Sponsor','TADIG', 'SPONSOR']], how='outer', on='TADIG', suffixes=['_CURRENT', '_NEW'])
    return df[(df.SPONSOR_CURRENT != df.SPONSOR_NEW)]

In [73]:
# Получение данных TADIG
# Получение прайсов с флагом is_discounted = 1

sql_srt=\
'''
SELECT DISTINCT c.COUNTRY_NAME,n.NETWORK_NAME,t.TADIG_CODE AS TADIG
,mcc.MCC
,t.NETWORK_ID FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
LEFT JOIN RDB_NETWORK_IMSI_PREFIXES mcc ON mcc.network_id = n.network_id
'''

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_tadig = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    
###################################################
# Adjustments
df_tadig.drop(
    df_tadig[(df_tadig.TADIG=='YUGPM') & (df_tadig.NETWORK_ID==623)].index, inplace=True)
# c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\PRICES\Partner_TURTK_YUGPM_not_discounted.msg
###################################################
    
sql_srt=\
'''
SELECT r.NETWORK_ID
,r.SPONSOR_ID AS SPONSOR
,r.data_rate
FROM ROAMING_PLAN_RULES r
WHERE r.roaming_plan_id IN (267,268,329,343,368,372)
--AND ((r.start_date < GETDATE() AND r.end_date is null) OR (r.start_date < GETDATE() AND r.end_date > GETDATE()))
AND r.end_date IS NULL
AND r.is_discounted = 1
'''

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_disc = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    
print('TADIGs and MCC with NETWORK_ID:')
display(df_tadig.head(3))

replace_sponsor_id_dict = dict((sponsor.sponsor_id, sponsor.sponsor) for sponsor in sponsors)
df_disc.SPONSOR = df_disc.SPONSOR.replace(replace_sponsor_id_dict)
print('\nDiscounted Networks:')
display(df_disc.head(3))
print('\nHigh cost in Discounted Networks:')
display(df_disc[df_disc.data_rate > 0.7])

TADIGs and MCC with NETWORK_ID:


,COUNTRY_NAME,NETWORK_NAME,TADIG,MCC,NETWORK_ID
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,412,4
1,Afghanistan,Areeba/MTN,AFGAR,412,5
2,Afghanistan,Etisalat,AFG55,412,6



Discounted Networks:


,NETWORK_ID,SPONSOR,data_rate
0,1208,S1,0.020
1,164,S4,0.005
2,499,S4,0.006



High cost in Discounted Networks:


,NETWORK_ID,SPONSOR,data_rate
568,360,S4,2.5805
589,1149,S2,1.3225


In [74]:
# Create df for existing "All" allowed lists 
# Create df for new "Allowed list"

# Like expression for Allowed List
df_al=pd.read_csv(join(downloads, dmi_allowed_list), usecols=['Sponsor','PLMN code'])
df_al.rename({'PLMN code':'TADIG'}, axis=1, inplace=True)

replace_dict = dict((sponsor.al_pattern, sponsor.sponsor) for sponsor in sponsors)

df_al['SPONSOR'] = df_al['Sponsor'].replace(to_replace=replace_dict, regex=True)

df_al = pd.merge(df_al, df_tadig, how='left', on='TADIG')

# Check not found Allowed Lists
print('Not found Allowed Lists:\n')
for sponsor in df_al.SPONSOR.unique().tolist():
    if sponsor not in [sponsor.sponsor for sponsor in sponsors]:
        print(sponsor)

df_al_s1 = df_al.loc[df_al.Sponsor.isin([s1_sponsor.al_all_name]), ['Sponsor', 'SPONSOR','NETWORK_ID','TADIG']]
df_al_s2 = df_al.loc[df_al.Sponsor.isin([s2_sponsor.al_all_name]), ['Sponsor', 'SPONSOR','NETWORK_ID','TADIG']]
df_al_s4 = df_al.loc[df_al.Sponsor.isin([s4_sponsor.al_all_name]), ['Sponsor', 'SPONSOR','NETWORK_ID','TADIG']]
df_al_s5 = df_al.loc[df_al.Sponsor.isin([s5_sponsor.al_all_name]), ['Sponsor', 'SPONSOR','NETWORK_ID','TADIG']]
df_al_s6 = df_al.loc[df_al.Sponsor.isin([s6_sponsor.al_all_name]), ['Sponsor', 'SPONSOR','NETWORK_ID','TADIG']]
df_al_s8 = df_al.loc[df_al.Sponsor.isin([s8_sponsor.al_all_name]), ['Sponsor', 'SPONSOR','NETWORK_ID','TADIG']]

df_al_s_list = [df_al_s1, df_al_s2, df_al_s4, df_al_s5, df_al_s6, df_al_s8]

for df_temp in df_al_s_list:
    display(df_temp.head(3))

df_new_al = pd.read_csv(join(downloads, file_new_al), sep=',')
df_new_al = pd.merge(df_new_al, df_tadig, how='left', on='TADIG')
df_new_al.drop_duplicates(inplace=True)
print('\nThe new allowed list uploaded from file enriched by MCC and NETWORK_ID:')
df_new_al.head(3)

Not found Allowed Lists:



,Sponsor,SPONSOR,NETWORK_ID,TADIG
50,Partner_All,S1,769,RUSNW
2585,Partner_All,S1,12,ALBVF
2586,Partner_All,S1,30,ARM01


,Sponsor,SPONSOR,NETWORK_ID,TADIG
90,P4_All,S2,10,ALBEM
651,P4_All,S2,45,AUSOP
701,P4_All,S2,103,BIHER


,Sponsor,SPONSOR,NETWORK_ID,TADIG
4280,MB_ALL,S4,996,AREDU
4281,MB_ALL,S4,49,AUSVF
4283,MB_ALL,S4,199,CHNCU


,Sponsor,SPONSOR,NETWORK_ID,TADIG
233,SMART,S5,12,ALBVF
638,SMART,S5,25,ARGCM
706,SMART,S5,27,ARG01


,Sponsor,SPONSOR,NETWORK_ID,TADIG
1301,S6_ALL,S6,769,RUSNW
1367,S6_ALL,S6,770,RUS01
1900,S6_ALL,S6,765,RUSBD


,Sponsor,SPONSOR,NETWORK_ID,TADIG
542,S8_ALL,S8,770,RUS01
637,S8_ALL,S8,20,AIACW
1715,S8_ALL,S8,10,ALBEM



The new allowed list uploaded from file enriched by MCC and NETWORK_ID:


,TADIG,SPONSOR,COUNTRY_NAME,NETWORK_NAME,MCC,NETWORK_ID
0,JAMDC,S5,Jamaica,Digicel Jamaica Limited,338,496
3,AUSVF,S5,Australia,Vodafone Hutchison Australia Pty Limited,505,49
4,AUSOP,S5,Australia,Singtel Optus,505,45


In [75]:
# Check mismatches with 'ALL' Allowed Lists in DMI and the new Combined Allowed Lists
# Prepare a file with TADIGs for creating Allowed List

def check_allowed(sponsor, df_new, df_allowed):
    df = pd.merge(df_new[df_new.SPONSOR == sponsor], df_allowed, how='left', on=['NETWORK_ID','SPONSOR'],
                      suffixes=['', '_AL'])
    return df

df_al_new_create = DataFrame()

for sponsor, df_al_s in zip(sponsors, df_al_s_list):
    print(f'Missing in {sponsor.al_all_name} Allowed list in DMI')
    df_temp_al = check_allowed(sponsor.sponsor, df_new_al, df_al_s)
    df_temp_missing = df_temp_al[df_temp_al.Sponsor.isnull()]
    display(df_temp_missing)
    df_temp_missing.to_csv(join(downloads, f'{sponsor.sponsor.lower()}_al_missing.csv'),index=False)
    df_al_new_create = df_al_new_create.append(df_temp_al, ignore_index=True, sort=False)

print(f'Create new Allowed list in DMI')
df_al_new_create.to_csv(join(downloads, f'df_al_create.csv'), index=False)
df_al_new_create.head(3)

Missing in Partner_All Allowed list in DMI


,TADIG,COUNTRY_NAME,NETWORK_NAME,MCC,Sponsor,SPONSOR,NETWORK_ID,TADIG_AL


Missing in P4_All Allowed list in DMI


,TADIG,COUNTRY_NAME,NETWORK_NAME,MCC,Sponsor,SPONSOR,NETWORK_ID,TADIG_AL


Missing in MB_ALL Allowed list in DMI


...

,TADIG,COUNTRY_NAME,NETWORK_NAME,MCC,Sponsor,SPONSOR,NETWORK_ID,TADIG_AL


Missing in S8_ALL Allowed list in DMI


,TADIG,COUNTRY_NAME,NETWORK_NAME,MCC,Sponsor,SPONSOR,NETWORK_ID,TADIG_AL


Create new Allowed list in DMI


,TADIG,COUNTRY_NAME,NETWORK_NAME,MCC,Sponsor,SPONSOR,NETWORK_ID,TADIG_AL
0,JAMDC,Jamaica,Digicel Jamaica Limited,338,NaN,S5,496,NaN
1,AUSVF,Australia,Vodafone Hutchison Australia Pty Limited,505,NaN,S5,49,NaN
2,AUSOP,Australia,Singtel Optus,505,SMART,S5,45,AUSOP


In [76]:
df_al_new_create.loc[df_al_new_create.TADIG_AL.notna(), ['TADIG_AL','Sponsor']].drop_duplicates()

,TADIG_AL,Sponsor
2,AUSOP,SMART
3,AUTON,SMART
5,BHRMV,SMART
7,CANTS,SMART
8,CANBM,SMART
...,...,...
390,GBROR,SMART
391,GBRJT,SMART
392,URYAM,SMART
393,UZBDU,SMART


In [69]:
# Check for different Sponsors in the same country

s1_temp = df_new_al.groupby('COUNTRY_NAME')['SPONSOR'].nunique()
s1_temp[s1_temp.values > 1]

Series([], Name: SPONSOR, dtype: int64)

In [77]:
# Check for not discounted in the new Allowed list

df_temp = pd.merge(df_new_al, df_disc, how='left', on=['NETWORK_ID', 'SPONSOR'])
df_temp[df_temp.data_rate.isnull()]

,TADIG,SPONSOR,COUNTRY_NAME,NETWORK_NAME,MCC,NETWORK_ID,data_rate
55,AIAK9,S5,NaN,NaN,NaN,NaN,NaN
56,ATGK9,S5,NaN,NaN,NaN,NaN,NaN
72,BMUK9,S5,NaN,NaN,NaN,NaN,NaN
74,BOLK9,S5,NaN,NaN,NaN,NaN,NaN
86,BFAK8,S5,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
348,VUTK8,S5,NaN,NaN,NaN,NaN,NaN
351,ZMBK8,S5,NaN,NaN,NaN,NaN,NaN
352,CAFK9,S5,NaN,NaN,NaN,NaN,NaN
353,GINK9,S5,NaN,NaN,NaN,NaN,NaN


### Prepare the TOC commands to create the full new Allowed list for Customer

In [63]:
# Create file with TOC commands to create the full new Allowed list for Customer

result_file_name_al_create = 'al_create.rcjson' # file name with TOC comands for cerating the new Allowed list
df_temp = df_al_new_create[['SPONSOR','TADIG_AL']].dropna().drop_duplicates()
prepare_toc_to_create_al(df_temp, downloads, result_file_name_al_create)

In [78]:
# Проверить различия с существующим Allowed list Кастомера.
# Проверяются только те операторы, которые были найдены в "All" Allowed list спонсоров.

df_al_custom_current = df_al.loc[df_al.Sponsor.isin([sponsor.al_name_combined_custom for sponsor in sponsors]),
                           ['Sponsor', 'TADIG', 'SPONSOR']]
df_al_custom_new = df_al_new_create[['Sponsor','SPONSOR','TADIG_AL']].rename({'TADIG_AL':'TADIG'}, axis=1).dropna().drop_duplicates()
df_make_changes_custom_current = check_al_for_differences(df_al_custom_current, df_al_custom_new)
df_make_changes_custom_current.to_csv(join(downloads, 'changes.csv'), index=False)
df_make_changes_custom_current

,Sponsor_CURRENT,TADIG,SPONSOR_CURRENT,Sponsor_NEW,SPONSOR_NEW


In [46]:
# Проверть, что нет несоответствия с Combined (после загрузки новых Allowed list)

df_al_comb = df_al.loc[df_al.Sponsor.isin([sponsor.al_name_combined_def for sponsor in sponsors]),
                       ['Sponsor','TADIG', 'SPONSOR']]
df_al_customer = df_al.loc[df_al.Sponsor.isin([sponsor.al_name_combined_custom for sponsor in sponsors]),
                           ['Sponsor','TADIG', 'SPONSOR']]
df_changes_comb_def = check_al_for_differences(df_al_comb, df_al_customer)
df_changes_comb_def

,Sponsor_CURRENT,TADIG,SPONSOR_CURRENT,Sponsor_NEW,SPONSOR_NEW
0,P4_Combined,HRVT2,S2,SMART,S5
1,P4_Combined,CZEET,S2,SMART,S5
2,P4_Combined,HRVVI,S2,SMART,S5
3,Partner_Combined,IDNTS,S1,SMART,S5
4,Partner_Combined,RUS01,S1,SMART,S5
...,...,...,...,...,...
261,NaN,CMRMT,NaN,SMART,S5
262,NaN,FJIDP,NaN,SMART,S5
263,NaN,MNGMN,NaN,SMART,S5
264,NaN,THADT,NaN,SMART,S5


### Prepare the TOC commands to adjust Allowed list for Customer

In [47]:
# Prepare commands to adjust the existing AL of customer by deleting the extras

delete_file_name = 'al_adjust_delete.rcjson' # file name with TOC comands for deleting the extras
df_temp = df_make_changes_custom_current[['Sponsor_CURRENT','TADIG']].dropna().drop_duplicates()
prepare_toc_to_delete_al(df_temp, downloads, delete_file_name)

In [19]:
# Prepare commands to adjust the existing AL of customer by createing missing operators in Allowed list

create_file_name = 'al_adjust_create.rcjson' # file name with TOC comands for creating missing operators in Allowed list
df_temp = df_make_changes_custom_current[['SPONSOR_NEW','TADIG']].rename({'SPONSOR_NEW':'SPONSOR'}, axis=1).dropna().drop_duplicates()
prepare_toc_to_create_al(df_temp, downloads, create_file_name)

### Prepare the TOC commands to create full new STS list for Customer

In [12]:
# Создать новые правила STS с нуля

result_file_name_sts_create = 'sts_create.rcjson'  # TOC comands for cerating STS list
df_temp = df_al_new_create[['SPONSOR','MCC','COUNTRY_NAME']].dropna().drop_duplicates()
prepare_toc_to_create_sts(df_temp, downloads, result_file_name_sts_create)

### Prepare the TOC commands to adjust STS list for Customer

In [11]:
# Проверить различия с существующим STS list Кастомера.

df_sts = pd.read_csv(join(downloads, dmi_sts_list))
df_sts.drop('Country name', axis=1, inplace=True)
df_sts.rename({'Sponsor profile':'STS_PROFILE_NAME'}, axis=1, inplace=True)

df_sts['SPONSOR'] = df_sts['STS_PROFILE_NAME'].replace(to_replace=replace_dict, regex=True)

df_sts_new = df_al_new_create[['SPONSOR','MCC','COUNTRY_NAME']].dropna().drop_duplicates()
df_sts_new.MCC = df_sts_new.MCC.astype('int64')

df_sts_customer = df_sts.loc[df_sts.STS_PROFILE_NAME.isin([s1_sponsor.sts_profile_name,
                                               s2_sponsor.sts_profile_name,
                                               s4_sponsor.sts_profile_name,
                                               s5_sponsor.sts_profile_name])]

df_sts_customer = pd.merge(df_sts_customer,
                 df_sts_new, how='outer', on='MCC',
                suffixes=['_CUSTOM_CURRENT', '_CUSTOM_NEW'])

df_make_changes = df_sts_customer[(df_sts_customer.SPONSOR_CUSTOM_CURRENT != df_sts_customer.SPONSOR_CUSTOM_NEW)]
df_make_changes

,STS_PROFILE_NAME,MCC,SPONSOR_CUSTOM_CURRENT,SPONSOR_CUSTOM_NEW,COUNTRY_NAME
40,Partner_DIALOQ,297,S1,S2,Montenegro
46,MB_DIALOQ,460,S4,S1,China
47,MB_DIALOQ,510,S4,S1,Indonesia
88,MB_DIALOQ,430,S4,NaN,NaN
89,MB_DIALOQ,431,S4,NaN,NaN
91,NaN,283,NaN,S1,Armenia
92,NaN,218,NaN,S1,Bosnia and Herzegov
93,NaN,370,NaN,S1,Dominican Republic
94,NaN,708,NaN,S1,Honduras
95,NaN,405,NaN,S1,India


In [12]:
# Prepare commands to adjust the existing STS list by deleting the extras

delete_file_name = 'sts_adjust_delete.rcjson'
df_temp = df_make_changes[['SPONSOR_CUSTOM_CURRENT','MCC','COUNTRY_NAME']].rename({'SPONSOR_CUSTOM_CURRENT':'SPONSOR'}, axis=1).dropna().drop_duplicates()
prepare_toc_to_delete_sts(df_temp, downloads, delete_file_name)

In [13]:
# Prepare commands to create missing countries in STS list

create_file_name = 'sts_adjust_create.rcjson'
df_temp = df_make_changes[['SPONSOR_CUSTOM_NEW','MCC','COUNTRY_NAME']].rename({'SPONSOR_CUSTOM_NEW':'SPONSOR'}, axis=1).dropna().drop_duplicates()
prepare_toc_to_create_sts(df_temp, downloads, create_file_name)

### После обновления AL обновить MCC-MAP файл

### Find Not Discounted in any Allowed Lists in DMI

In [5]:
# Find Not Discounted in any Allowed Lists in DMI and prepare TOC commands to delete

df_temp = pd.merge(df_al, df_disc, how='left', on=['NETWORK_ID','SPONSOR'])
df_temp = df_temp[df_temp.data_rate.isnull()]
df_temp.to_csv(join(downloads, out_file_not_disc), index=False)

# Prepare commands to adjust the existing AL of customer by deleting the extras
prepare_toc_to_delete_al(df_temp[['Sponsor','TADIG']].dropna().drop_duplicates(), downloads, out_file_toc_al_delete_not_disc)

df_temp

,Sponsor,TADIG,SPONSOR,COUNTRY_NAME,NETWORK_NAME,MCC,NETWORK_ID,data_rate
1186,P4_TCOM_COMB,BGRMT,S2,NaN,NaN,NaN,NaN,NaN
1301,S6_ALL,RUSNW,S6,Russian Federation,OJSC MegaFon,250,769,NaN
1367,S6_ALL,RUS01,S6,Russian Federation,PJSC Mobile TeleSystems MTS,250,770,NaN
1900,S6_ALL,RUSBD,S6,Russian Federation,VimpelCom PJSC,250,765,NaN


### Find Missing TADIGs in DMI

In [22]:
# Find Missing TADIGs in DMI

create_file_missing_operators_dmi = 'missing_operators_dmi.csv'

list_operators_dmi = []
for df_temp in df_al_s_list:
    list_operators_dmi.extend(df_temp.NETWORK_ID.astype('int').unique().tolist())
    
df_temp = pd.merge(df_disc, df_tadig, how='left', on='NETWORK_ID', suffixes=['_AL',''])\
    [['COUNTRY_NAME','NETWORK_NAME','TADIG','MCC','NETWORK_ID']].drop_duplicates()
df_missing_operators_dmi = df_temp[~df_temp.NETWORK_ID.isin(list_operators_dmi)]
df_missing_operators_dmi.to_csv(join(downloads, create_file_missing_operators_dmi), index=False)
df_missing_operators_dmi.head()

,COUNTRY_NAME,NETWORK_NAME,TADIG,MCC,NETWORK_ID
24,Zimbabwe,Econet Wireless Private Limited,ZWEET,648,1144
26,Malawi,Airtel Malawi Limited,MWICT,650,577
31,Namibia,Leo / Orascom,NAM03,649,634
32,Niger,Orange/Sahelc.,NEROR,614,675
34,Nigeria,MTN,NGAMN,621,681


### Developments

#### Create 'ALL' allowed list

In [61]:
df_tadig_in_all_al = DataFrame()
for df_temp in df_al_s_list:
    df_tadig_in_all_al = df_tadig_in_all_al.append(df_temp[['NETWORK_ID','TADIG']], ignore_index=True, sort=False)

df_tadig_in_all_al = df_tadig_in_all_al.drop_duplicates()

df_temp = pd.merge(df_al_new_create.loc[df_al_new_create.TADIG_AL.isna()],
                   df_tadig_in_all_al, how='left', on='NETWORK_ID', suffixes=['_OCS',''])

# Check for not discounted in the new Allowed list

df_temp = pd.merge(df_temp, df_disc, how='left', on=['NETWORK_ID', 'SPONSOR'])
df_temp = df_temp[~df_temp.data_rate.isnull()]

df_temp = df_temp.dropna(subset=['TADIG'])[['COUNTRY_NAME','NETWORK_NAME','SPONSOR','TADIG','TADIG_OCS','data_rate']]

df_temp.to_csv(join(downloads, 'al_create_s8.csv'), index=False)

display(df_temp.head())

prepare_toc_to_create_al(df_temp[['SPONSOR','TADIG']].dropna().drop_duplicates(), downloads, 'al_create_s8.rcjson')

,COUNTRY_NAME,NETWORK_NAME,SPONSOR,TADIG,TADIG_OCS,data_rate
2,Anguilla,Cable and Wireless,S8,AIACW,AIACW,0.1260
3,Albania,Eagle Mobile,S8,ALBEM,ALBEM,0.0636
4,Albania,Vodafone,S8,ALBVF,ALBVF,0.0204
6,United Arab Emirates,Emirates Integrated Telecommunications Company...,S8,AREDU,AREDU,0.3060
7,Argentina,AMX Argentina S A,S8,ARGCM,ARGCM,0.5328
